### Clustering using Hash functions
Trying to implement strand isolation using purely Hash functions. Want to select the same strands together after signatures using some similarity metric

In [1]:
%load_ext autoreload
%autoreload 2

In [48]:
import numpy as np
from aligned_clustering import conduct_align_clustering
from utils import get_fastq_records, load_json_file, get_original_strands, get_badread_strand_id, get_recovery_percentage
import Levenshtein
import random
from Levenshtein import ratio
from collections import Counter

In [32]:
#records_original = get_fastq_records(r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\runs\2025-01-30 21.21.00.463318\reads_no_adapters.fastq")
records_original = get_fastq_records(r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\runs\2025-01-30 21.21.00.463318\reads_0.98.fastq")

In [33]:
original_strand_ids, coupling_rates, capping_flags, original_strands = get_original_strands(r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\runs\2025-01-30 21.21.00.463318\original_strands.txt")

strand_ids_synthesized = load_json_file(r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\runs\2025-01-30 21.21.00.463318\synthesized_uid_reference.json")

In [34]:
records = [i for i in records_original if get_badread_strand_id(i) in strand_ids_synthesized]
sequenced_strands = [str(i.seq) for i in records]
strand_ids_sequenced = [get_badread_strand_id(i) for i in records]

Levenshtien edit distance works pretty well. That's my metric. Kmeans works for now. I'll bring down junk reads etc, remove adapters, and test it again to see how well aggregation works.
Maybe try DBscan with Levenshtien distance

In [74]:
pools = {i: [] for i in strand_ids_sequenced}

for i in range(len(sequenced_strands)):
    for j in range(len(sequenced_strands)):
        if not i==j:
            if ratio(sequenced_strands[i], sequenced_strands[j]) > 0.95:
                pools[strand_ids_sequenced[i]].append(strand_ids_sequenced[j]) 


In [75]:
# We now have the pairwise pools
els = []

for strand_id in pools:
    els.extend(pools[strand_id])
    for strand in pools[strand_id]:
        els.extend(pools[strand])

In [59]:
print(els)

['a4c10e8a-3045-4237-a885-fddd5b7f3819', 'c2e27a94-eac9-4cea-9ac5-98ba051236d7', 'ad1944a8-e640-4dec-a29f-5d50d8c6f595', '41f706b9-e601-438f-91ba-b8e58b5cb67a', '080a78f9-1528-42a5-ac6a-e900541bdcc9', '18a01502-aa11-4d8e-9494-bff2f6a335dc', '72152731-2cb9-42ec-96b1-23042046adda', '2a85710f-751c-4d5c-9304-29d536d7730d', '728f74f4-be75-4cc5-8d25-7712ee4585cb', 'cf1e0c01-df78-48fa-aa27-9c7fcd25498a', 'f74414ce-f7b5-4c08-b1d6-91a7b0763683', 'd5dfbe7b-39ac-42ba-9e64-571baf1ce854', '8ef7df2f-2c17-48ab-b129-6c56d69d1894', '80f279f6-0070-4bdc-b4e7-80a88784ceb9', '23b691fb-2801-4740-870f-80450e1b4efd', '3554ec5a-a737-4ea2-bef6-6158932b10c6', '58f52563-190f-43fb-a967-3878ccf7fad3', 'ec0ad9f1-ec6b-4683-afa6-de9d16f6ce51', 'cacdfcf9-d3ec-4c34-af35-35c7948dd752', '86954dfd-7e1c-47d6-88bf-d30546d93cf2', '4bcc9a3f-f581-4b3a-b5e0-5a56161984ef', '2ff6f2d4-9e00-4aaa-a7cc-c1a0aabb1003', '25ce8541-a9f6-46c2-86a5-faf9c05abfc8', 'b7d71892-3e83-4da7-a54e-5fd49084361e', '7c0eed9d-3666-4cd3-93bc-b0926a946a56',

In [ ]:
from sklearn_extra.cluster import KMedoids

# Choose the number of clusters (k)
k = 3

# Run k-medoids
kmedoids = KMedoids(n_clusters=k, metric="precomputed", random_state=42)
labels = kmedoids.fit_predict(levenshtein_matrix)

print(labels)  # Cluster assignments